<a href="https://colab.research.google.com/github/kiplangatkorir/Simple-LLM/blob/main/SimpleLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

class SimpleDataset(Dataset):
    def __init__(self, text, seq_length):
        self.text = text
        self.seq_length = seq_length
        self.vocab = sorted(set(text))
        self.char2idx = {c: i for i, c in enumerate(self.vocab)}
        self.idx2char = {i: c for i, c in enumerate(self.vocab)}
        self.encoded = [self.char2idx[c] for c in text]

    def __len__(self):
        return len(self.encoded) - self.seq_length

    def __getitem__(self, idx):
        x = self.encoded[idx:idx + self.seq_length]
        y = self.encoded[idx + 1:idx + self.seq_length + 1]
        return torch.tensor(x), torch.tensor(y)

class SimpleLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(SimpleLSTM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        x, hidden = self.lstm(x, hidden)
        x = self.fc(x)
        return x, hidden

# Hyperparameters
text = "Once upon a time in a land far, far away, there lived a wise old king. This king ruled over a prosperous kingdom filled with lush forests, sparkling rivers, and golden fields. The people of the kingdom were happy and content, for the king was kind and just. Every day, he would sit upon his throne and listen to the needs and concerns of his people.\nOne day, a young traveler arrived at the palace gates. He carried a strange artifact that he claimed could reveal the future. The king, curious but cautious, invited the traveler to demonstrate this artifact's powers...\n"
seq_length = 50
batch_size = 32
embed_dim = 64
hidden_dim = 128
epochs = 20

dataset = SimpleDataset(text, seq_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = SimpleLSTM(len(dataset.vocab), embed_dim, hidden_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
for epoch in range(epochs):
    hidden = None
    for x, y in dataloader:
        optimizer.zero_grad()
        output, hidden = model(x)
        loss = criterion(output.view(-1, len(dataset.vocab)), y.view(-1))
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

# Text Generation
def generate_text(model, start_str, length=100):
    model.eval()
    chars = [dataset.char2idx[c] for c in start_str]
    input_seq = torch.tensor(chars).unsqueeze(0)
    hidden = None
    generated = start_str

    for _ in range(length):
        output, hidden = model(input_seq, hidden)
        char_idx = torch.argmax(output[:, -1, :]).item()
        generated += dataset.idx2char[char_idx]
        input_seq = torch.tensor([[char_idx]])

    return generated

print(generate_text(model, start_str="The ", length=200))

Epoch 1, Loss: 2.9607
Epoch 2, Loss: 2.6111
Epoch 3, Loss: 2.1844
Epoch 4, Loss: 1.9966
Epoch 5, Loss: 1.6182
Epoch 6, Loss: 1.4319
Epoch 7, Loss: 1.1322
Epoch 8, Loss: 0.9247
Epoch 9, Loss: 0.7245
Epoch 10, Loss: 0.6320
Epoch 11, Loss: 0.5237
Epoch 12, Loss: 0.4015
Epoch 13, Loss: 0.3568
Epoch 14, Loss: 0.2897
Epoch 15, Loss: 0.2436
Epoch 16, Loss: 0.2513
Epoch 17, Loss: 0.2161
Epoch 18, Loss: 0.2285
Epoch 19, Loss: 0.1989
Epoch 20, Loss: 0.1883
The king was kind and just. Every day, he would sit upon his throne and listen to the needs and concerns of his people.
One day, a young traveler arrived at the palace gates. He carried a strange artifact
